# Cleaning Up the Price Data

Goals:

1.  Working with original `CSV` files
2.  Convert the `item-price` column from a string to a float
3.  Plot to confirm pre-processing worked

In [2]:
import pandas as pd
import os

## Read All CSV Files Into A Dataframe

In [3]:
data_folder = os.path.join('..', 'data', 'waitrose-2024-07')

In [ ]:
# Use list comprehension to collect all the files in the folder and create their folder path
all_files = [
    os.path.join(data_folder, file) for file in os.listdir(data_folder)
    if file.endswith('.csv')

]

In [9]:
for file in all_files:
    print(file)

..\data\waitrose-2024-07\baby-child-and-parent.csv
..\data\waitrose-2024-07\bakery.csv
..\data\waitrose-2024-07\beer-wine-and-spirits.csv
..\data\waitrose-2024-07\best-of-british.csv
..\data\waitrose-2024-07\dietary-and-lifestyle.csv
..\data\waitrose-2024-07\everyday-value.csv
..\data\waitrose-2024-07\food-cupboard.csv
..\data\waitrose-2024-07\fresh-and-chilled.csv
..\data\waitrose-2024-07\frozen.csv
..\data\waitrose-2024-07\home.csv
..\data\waitrose-2024-07\household.csv
..\data\waitrose-2024-07\new.csv
..\data\waitrose-2024-07\organic-shop.csv
..\data\waitrose-2024-07\pet.csv
..\data\waitrose-2024-07\summer.csv
..\data\waitrose-2024-07\tea-coffee-and-soft-drinks.csv
..\data\waitrose-2024-07\toiletries-health-and-beauty.csv
..\data\waitrose-2024-07\waitrose-brands.csv


In [ ]:
# Read every single file as a DataFrame